In [24]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler

In [25]:
class NaiveDense:
  def __init__(self, input_size, output_size, activation):
    self.activation = activation

    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
    self.W = tf.Variable(w_initial_value)

    b_shape = (output_size, )
    b_initial_value = tf.random.uniform(b_shape, minval=0, maxval=1e-1)
    self.b = tf.Variable(b_initial_value)

  def __call__(self, x):
    return self.activation(tf.matmul(x, self.W) + self.b)

  @property
  def weights(self):
    return [self.W, self.b]

In [26]:
class NaiveSequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights +=  layer.weights
    return weights


In [27]:
model = NaiveSequential([
    NaiveDense(input_size=28*28, output_size=100, activation=tf.nn.relu),
    NaiveDense(input_size=100, output_size=50, activation=tf.nn.gelu),
    NaiveDense(input_size=50, output_size=10, activation=tf.nn.softmax),
])
print(model.weights)

[<tf.Variable 'Variable:0' shape=(784, 100) dtype=float32, numpy=
array([[0.09162509, 0.02991783, 0.06724264, ..., 0.04947385, 0.00553541,
        0.07803314],
       [0.083667  , 0.01264511, 0.04550705, ..., 0.03265628, 0.07768156,
        0.07385997],
       [0.00664485, 0.06013179, 0.04018882, ..., 0.08683378, 0.05298656,
        0.0144342 ],
       ...,
       [0.00614295, 0.0385685 , 0.06606881, ..., 0.04629766, 0.08606388,
        0.08543887],
       [0.04041293, 0.0498327 , 0.0518841 , ..., 0.06428986, 0.03414806,
        0.0706495 ],
       [0.01906486, 0.06660533, 0.0490945 , ..., 0.09290172, 0.04867165,
        0.02983735]], dtype=float32)>, <tf.Variable 'Variable:0' shape=(100,) dtype=float32, numpy=
array([0.01635808, 0.07966319, 0.07765136, 0.02601728, 0.07935145,
       0.01860422, 0.09773003, 0.00204278, 0.06383976, 0.00791686,
       0.07461329, 0.03417253, 0.040545  , 0.01173204, 0.02852907,
       0.00495479, 0.08301234, 0.00118067, 0.0020188 , 0.09769528,
       0.02

In [28]:
import math
class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)

  def next(self):
    images = self.images[self.index:self.index+self.batch_size]
    labels = self.labels[self.index:self.index+self.batch_size]
    self.index += self.batch_size
    return images, labels

In [29]:
from tensorflow.keras import optimizers
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
def update_weights(gradients, weights):
  optimizer.apply_gradients(zip(gradients, weights))

In [30]:
def one_training_step(model, images_batch, labels_batch):
  with tf.GradientTape() as tape:
    predictions = model(images_batch)
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
    average_loss = tf.reduce_mean(per_sample_losses)
  gradients = tape.gradient(average_loss, model.weights)
  update_weights(gradients, model.weights)
  return average_loss

In [31]:
def fit(model, images, labels, epochs, batch_size=128):
  for epoch_number in range(epochs):
    print(f"Epoch number {epoch_number}:")
    batch = BatchGenerator(images, labels, batch_size)
    for batch_number in range(batch.num_batches):
      images_batch, labels_batch = batch.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_number % 100 == 0:
        print(f"Batch loss {batch_number}: {loss:.2f}")

In [32]:
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import StandardScaler
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], train_images.shape[1]*train_images.shape[2])

test_images = test_images.reshape(test_images.shape[0], test_images.shape[1]*test_images.shape[2])

sc = StandardScaler()
train_images_std = sc.fit_transform(train_images)
test_images_std = sc.transform(test_images)
train_images_std = tf.convert_to_tensor(train_images_std, tf.float32)
test_images_std = tf.convert_to_tensor(test_images_std, tf.float32)
print(train_images_std)

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(60000, 784), dtype=float32)


In [33]:
fit(model, train_images_std, train_labels, epochs=15, batch_size=200)

Epoch number 0:


Batch loss 0: 5.03
Batch loss 100: 1.31
Batch loss 200: 0.66
Epoch number 1:
Batch loss 0: 0.41
Batch loss 100: 0.29
Batch loss 200: 0.19
Epoch number 2:
Batch loss 0: 0.14
Batch loss 100: 0.18
Batch loss 200: 0.13
Epoch number 3:
Batch loss 0: 0.11
Batch loss 100: 0.14
Batch loss 200: 0.09
Epoch number 4:
Batch loss 0: 0.09
Batch loss 100: 0.11
Batch loss 200: 0.07
Epoch number 5:
Batch loss 0: 0.08
Batch loss 100: 0.10
Batch loss 200: 0.06
Epoch number 6:
Batch loss 0: 0.07
Batch loss 100: 0.09
Batch loss 200: 0.05
Epoch number 7:
Batch loss 0: 0.06
Batch loss 100: 0.07
Batch loss 200: 0.04
Epoch number 8:
Batch loss 0: 0.05
Batch loss 100: 0.06
Batch loss 200: 0.04
Epoch number 9:
Batch loss 0: 0.04
Batch loss 100: 0.05
Batch loss 200: 0.05
Epoch number 10:
Batch loss 0: 0.03
Batch loss 100: 0.06
Batch loss 200: 0.02
Epoch number 11:
Batch loss 0: 0.03
Batch loss 100: 0.03
Batch loss 200: 0.03
Epoch number 12:
Batch loss 0: 0.03
Batch loss 100: 0.02
Batch loss 200: 0.02
Epoch number

In [34]:
predictions = model(test_images_std)
predictions = np.argmax(predictions.numpy(), axis=1)
matches = predictions == test_labels
print(f"Accuracy: {matches.mean():.2f}")

Accuracy: 0.97
